<a href="https://colab.research.google.com/github/TMUITLab/EAFR/blob/main/mytest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
%cd '/content'
!git clone https://github.com/1049451037/GCN-Align
!git clone https://github.com/MaoXinn/RREA
with open('/content/rel_ids_1') as f1:
  rels1 = f1.readlines()

with open('/content/rel_ids_2') as f1:
  rels2 = f1.readlines()

with open('/content/GCN-Align/data/zh_en/ent_ids_1') as f1:
  ents1 = f1.readlines()

with open('/content/GCN-Align/data/zh_en/ent_ids_2') as f1:
  ents2 = f1.readlines()
rel_size = 2 * 3024

/content
fatal: destination path 'GCN-Align' already exists and is not an empty directory.
fatal: destination path 'RREA' already exists and is not an empty directory.


In [5]:
rels1_dict = dict()
rels2_dict = dict()
ent1_dict = dict()
ent2_dict = dict()
rrels_dict = dict()
rent_dict = dict()

for line in rels1:
  if len(line) > 0:
    last_part = line[line.rindex('/')+1:-1].lower()
    if last_part not in rels1_dict:
      rels1_dict[last_part] = [int(line.split('\t')[0])]
      
    else:
      rels1_dict[last_part] += [int(line.split('\t')[0])]
  rrels_dict[int(line.split('\t')[0]) ] = last_part
  rrels_dict[int(line.split('\t')[0]) + rel_size//2] = last_part

for line in rels2:
  if len(line) > 0:
    last_part = line[line.rindex('/')+1:-1].lower()
    if last_part not in rels2_dict:
      rels2_dict[last_part] = [int(line.split('\t')[0])]
    else:
      rels2_dict[last_part] += [int(line.split('\t')[0])]
  rrels_dict[int(line.split('\t')[0])  ] = last_part
  rrels_dict[int(line.split('\t')[0]) + rel_size//2 ] = last_part

for line in ents1:
  if len(line) > 0:
    last_part = line[line.rindex('/')+1:-1].lower()
    if last_part not in ent1_dict:
      ent1_dict[last_part] = [int(line.split('\t')[0])]
      
    else:
      ent1_dict[last_part] += [int(line.split('\t')[0])]
  rent_dict[int(line.split('\t')[0]) ] = last_part

for line in ents2:
  if len(line) > 0:
    last_part = line[line.rindex('/')+1:-1].lower()
    if last_part not in ent2_dict:
      ent2_dict[last_part] = [int(line.split('\t')[0])]
    else:
      ent2_dict[last_part] += [int(line.split('\t')[0])]
  rent_dict[int(line.split('\t')[0])  ] = last_part

In [55]:
relation_maps1 = dict()
relation_maps2 = dict()
for line in rels1:
  if len(line) > 0:
    last_part = line[line.rindex('/')+1:-1].lower()
    if last_part  in rels2_dict:
      relation_maps1[int(line.split('\t')[0])] = rels2_dict[last_part]

for line in rels2:
  if len(line) > 0:
    last_part = line[line.rindex('/')+1:-1].lower()
    if last_part  in rels1_dict:
      relation_maps2[int(line.split('\t')[0])] = rels1_dict[last_part]


In [7]:
import warnings
warnings.filterwarnings('ignore')

from importlib.machinery import SourceFileLoader
from google.colab import files

layer = SourceFileLoader("layer", "/content/RREA/CIKM/layer.py").load_module()
utils = SourceFileLoader("utils", "/content/RREA/CIKM/utils.py").load_module()
CSLS = SourceFileLoader("CSLS", "/content/RREA/CIKM/CSLS.py").load_module()
import tensorflow as tf
import os
import time
import random
import keras
import pickle
from scipy.sparse import hstack
import math
from tqdm import *
import numpy as np
from utils import *
from CSLS import *
import tensorflow as tf
import keras.backend as K
from keras.layers import *
from layer import NR_GraphAttention

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [27]:
lang = 'zh'
radj,train_pair,dev_pair,adj_matrix,r_index,r_val,adj_features,rel_features,nadj,char_size = load_data('/content/GCN-Align/data/%s_en/'%lang,train_ratio=0.30)
train_pair_main=train_pair
adj_matrix = np.stack(adj_matrix.nonzero(),axis = 1)
rel_matrix,rel_val = np.stack(rel_features.nonzero(),axis = 1),rel_features.data
ent_matrix,ent_val = np.stack(adj_features.nonzero(),axis = 1),adj_features.data

38960 3024


In [28]:
rel_size = rel_features.shape[1] 
rel_matrix1 = rel_matrix.copy()
for i in range(len(rel_matrix1)):
  rel_matrix1[i,1] = (rel_matrix1[i,1] + rel_size//2) % rel_size

radj1 = radj.copy()
for i in range(len(radj1)):
  radj1[i,2] = (radj1[i,2] + rel_size//2) % rel_size
single_triple_set = set()
degree = np.zeros([adj_features.shape[0],3])
for triple in radj:
    single_triple_set.add((triple[0],triple[1]))
    if triple[2] < rel_size //2:
        degree[triple[0],0] += 1
        degree[triple[1],0] += 1
        degree[triple[0],2] += 1
        degree[triple[1],1] += 1

dev_pair_degrees = np.zeros([2,len(dev_pair),3])
for i in range(len(dev_pair)):
    dev_pair_degrees[0,i,0] = degree[dev_pair[i][0],0]
    dev_pair_degrees[0,i,1] = degree[dev_pair[i][0],1]
    dev_pair_degrees[0,i,2] = degree[dev_pair[i][0],2]
    dev_pair_degrees[1,i,0] = degree[dev_pair[i][1],0]
    dev_pair_degrees[1,i,1] = degree[dev_pair[i][1],1]
    dev_pair_degrees[1,i,2] = degree[dev_pair[i][1],2]

train_pair_degrees = np.zeros([2,len(train_pair),3])
for i in range(len(train_pair)):
    train_pair_degrees[0,i,0] = degree[train_pair[i][0],0]
    train_pair_degrees[0,i,1] = degree[train_pair[i][0],1]
    train_pair_degrees[0,i,2] = degree[train_pair[i][0],2]
    train_pair_degrees[1,i,0] = degree[train_pair[i][1],0]
    train_pair_degrees[1,i,1] = degree[train_pair[i][1],1]
    train_pair_degrees[1,i,2] = degree[train_pair[i][1],2]

ent_size = adj_features.shape[0]
depth = 3
deg = np.zeros([ent_size])
deg_in = np.zeros([ent_size])
deg_out = np.zeros([ent_size])
bdeg = np.zeros([ent_size])


single_graph = dict()
for h,t,r in radj:
    deg[h] += 1
    deg[t] += 1
    deg_in[t] += 1
    deg_out[h] += 1
    if(h not in single_graph):
        single_graph[h] = [t]
    else:
        single_graph[h] += [t]
for i in range(ent_size):
    bdeg[i] = math.floor(math.log(i+1))

bin = int(max(bdeg))
rec_deg = np.zeros([ent_size,bin*depth])
for h in single_graph:
    set_len = len(single_graph[h])
    for i in single_graph[h]:
        rec_deg[h,int(bdeg[i])] += 1/set_len
for d in range(1,depth):
    
    for h in single_graph:
        set_len = len(single_graph[h])
        for i in single_graph[h]:
            rec_deg[h,d*bin:(d+1)*bin] += (rec_deg[i,(d-1)*bin:(d)*bin])/set_len


In [18]:
entity1, rel1, triples1 = load_triples('/content/GCN-Align/data/%s_en/'%lang + 'triples_1')
num_entity_1 = len(entity1)
num_rel_1 = len(rel1)

In [33]:
rmap_predict = dict()
train_pair_dic = dict()
pairs = np.concatenate((dev_pair , train_pair),axis=0)
for i in range(len(train_pair)):
  train_pair_dic[train_pair[i][0]] = train_pair[i][1]
triple_dic = dict()
for h,t,r in radj:
    if r < rel_size //2 :
      if (h,t) in triple_dic:
        triple_dic[(h,t)] += [r]
      else:
        triple_dic[(h,t)] = [r]
for h,t,r in radj:
  if h in train_pair_dic and t in train_pair_dic and h != t and r < rel_size //2:
    z = np.zeros([rel_size])
    if (train_pair_dic[h],train_pair_dic[t]) in triple_dic:
      z[triple_dic[(train_pair_dic[h],train_pair_dic[t])]] = 1
      if r in rmap_predict :
        rmap_predict[r] += z
      else:
        rmap_predict[r] = z
print(rmap_predict)

{652: array([0., 0., 0., ..., 0., 0., 0.]), 968: array([0., 0., 0., ..., 0., 0., 0.]), 37: array([0., 0., 0., ..., 0., 0., 0.]), 815: array([0., 0., 0., ..., 0., 0., 0.]), 1137: array([0., 0., 0., ..., 0., 0., 0.]), 1491: array([0., 0., 0., ..., 0., 0., 0.]), 657: array([0., 0., 0., ..., 0., 0., 0.]), 78: array([0., 0., 0., ..., 0., 0., 0.]), 630: array([0., 0., 0., ..., 0., 0., 0.]), 290: array([0., 0., 0., ..., 0., 0., 0.]), 831: array([0., 0., 0., ..., 0., 0., 0.]), 260: array([0., 0., 0., ..., 0., 0., 0.]), 471: array([0., 0., 0., ..., 0., 0., 0.]), 808: array([0., 0., 0., ..., 0., 0., 0.]), 271: array([0., 0., 0., ..., 0., 0., 0.]), 351: array([0., 0., 0., ..., 0., 0., 0.]), 1421: array([0., 0., 0., ..., 0., 0., 0.]), 1003: array([0., 0., 0., ..., 0., 0., 0.]), 1026: array([0., 0., 0., ..., 0., 0., 0.]), 569: array([0., 0., 0., ..., 0., 0., 0.]), 414: array([0., 0., 0., ..., 0., 0., 0.]), 447: array([0., 0., 0., ..., 0., 0., 0.]), 1441: array([0., 0., 0., ..., 0., 0., 0.]), 533: a

In [45]:
i = 0
for key in rmap_predict:
  v = np.max(rmap_predict[key])
  i += 1
  if(v>50):
    a = np.argmax(rmap_predict[key])
    print(key,np.argmax(rmap_predict[key]),rrels_dict[a],rrels_dict[key],v)
  #print(key,np.argmax(rmap_predict[key]),np.max(rmap_predict[key]) , )



652 2133 reportsto 政治派系 193.0
471 2167 parts sponsors 59.0
271 2152 establisheddate 聲優 304.0
476 2172 start majorapplications 132.0
494 2152 establisheddate 相关剧集 67.0
634 2189 blank4infosec 职位 56.0
819 2227 cultures 啟始客戶 185.0
21 2160 language regionalname 152.0
177 2178 weather method 52.0
388 2180 royalanthem application 63.0
463 2200 nobellaureates 旗下數位頻道 57.0


In [46]:
r1 = 471  
r2 = 2167  
print (rrels_dict[r1],rrels_dict[r2])
i = 0
for h,t,r in radj:
  
  if h in train_pair_dic and t in train_pair_dic and h != t : 
    if(r == r1 and (train_pair_dic[h],train_pair_dic[t]) in triple_dic and r2 in triple_dic[(train_pair_dic[h],train_pair_dic[t])]):
      print(h,t,rent_dict[h],rent_dict[t],rent_dict[train_pair_dic[h]],rent_dict[train_pair_dic[t]])
      print(h,train_pair_dic[h],t,train_pair_dic[t])
      i += 1
  # elif (r == r1 and h != t):
  #    print(h,t,rent_dict[h],rent_dict[t]) 
print(i)   

sponsors parts
100 10241 布兰妮_(专辑) 別讓我最後知道 britney_(album) don't_let_me_be_the_last_to_know
100 10600 10241 20741
100 22783 布兰妮_(专辑) 少女已過，熟女未滿 britney_(album) i'm_not_a_girl,_not_yet_a_woman
100 10600 22783 37834
1148 1106 威廉·本廷克勋爵 印度总督 lord_william_bentinck governor-general_of_india
1148 11648 1106 11606
1600 3442 维多利亚_(英国君主) 印度皇帝 queen_victoria emperor_of_india
1600 12100 3442 13942
1726 23464 阿方索·卡洛斯_(聖海梅公爵) 西班牙君主列表 infante_alfonso_carlos,_duke_of_san_jaime list_of_spanish_monarchs
1726 12226 23464 35462
2201 5539 this_is_it_(迈克尔·杰克逊专辑) black_or_white michael_jackson's_this_is_it_(album) black_or_white
2201 12701 5539 16039
2201 10463 this_is_it_(迈克尔·杰克逊专辑) billie_jean michael_jackson's_this_is_it_(album) billie_jean
2201 12701 10463 20963
2529 5621 切斯特·威廉·尼米兹 時代_(雜誌) chester_w._nimitz time_(magazine)
2529 13029 5621 16121
2683 5992 扬·卡齐米日 波兰君主列表 john_ii_casimir_vasa list_of_polish_monarchs
2683 13183 5992 16492
2901 6326 第二代羅金漢侯爵查爾斯·沃森-文特沃斯 上議院領袖 charles_watson-wentworth,_2nd_marque

In [36]:
for h,t,r in radj:
  if (h== 10617   and t == 11315):
    print(r,rrels_dict[r])

2152 establisheddate


In [26]:
for i in range(rel_size):
  if(rmap_predict[652][i] > 0):
    print(i,rmap_predict[652][i])


1708 1.0
1712 1.0
1722 1.0
1726 1.0
1767 1.0
1774 5.0
1776 2.0
1871 1.0
1912 1.0
2002 1.0
2051 1.0
2133 2122.0
2134 70.0
2137 6.0
2147 742.0
2159 1.0
2164 15.0
2165 1.0
2167 2.0
2168 19.0
2176 18.0
2179 3.0
2184 2.0
2189 155.0
2200 134.0
2217 133.0
2225 3.0
2227 198.0
2243 1.0
2244 14.0
2267 2.0
2270 6.0
2271 1.0
2277 2.0
2317 9.0
2327 3.0
2341 1.0
2361 6.0
2368 14.0
2379 12.0
2388 27.0
2390 1.0
2415 1.0
2481 1.0
2524 2.0
2529 1.0
2538 2.0
2544 10.0
2562 3.0
2569 1.0
2584 2.0
2586 3.0
2596 1.0
2849 1.0
2889 1.0


In [108]:
print(np.max(train_pair[:,0]),np.max(train_pair[:,1]))
print(np.min(train_pair[:,0]),np.min(train_pair[:,1]))

25499 38959
0 10500


In [109]:
ss = dict()

for h,r,t in radj:
  if h in train_pair_dic and t in train_pair_dic and h!=t:
    
    ss[(h,t)] = 1
print(ss.keys())


dict_keys([(0, 652), (0, 3557), (0, 3843), (0, 3487), (0, 819), (0, 79), (0, 3676), (0, 21), (1, 3487), (1, 79), (1, 3843), (1, 652), (1, 388), (1, 3676), (1, 819), (2, 18), (2, 786), (2, 452), (3, 1198), (3, 3201), (3, 495), (3, 3544), (3, 670), (4, 1320), (5, 3614), (5, 997), (5, 590), (5, 3557), (6, 3295), (6, 3438), (6, 3048), (6, 177), (6, 3126), (6, 3518), (6, 3419), (7, 451), (7, 999), (7, 477), (7, 174), (8, 3208), (8, 762), (8, 697), (9, 526), (9, 3487), (9, 652), (9, 3843), (10, 3630), (10, 1067), (10, 3198), (10, 1208), (10, 4048), (10, 1024), (10, 1011), (10, 1127), (11, 533), (11, 176), (11, 827), (11, 3557), (11, 590), (12, 271), (12, 414), (12, 1244), (12, 476), (12, 494), (13, 271), (13, 1148), (14, 3992), (14, 968), (14, 1462), (15, 3562), (15, 3285), (15, 126), (15, 388), (16, 3763), (16, 692), (17, 21), (17, 634), (17, 3557), (17, 652), (17, 388), (17, 3843), (17, 463), (17, 374), (18, 819), (18, 652), (18, 3843), (18, 199), (18, 436), (18, 827), (18, 808), (18, 630)

In [ ]:
for h,r,t in radj:
  print(t)
  if(t == 19238):
    print(h,r,t)

In [78]:
print(ss.keys())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
train_pair_dic

In [68]:
max(train_pair_dic.values())

38896